In [38]:
import json
with open('ebird_config.json') as f:
    data = json.load(f)
    recent_species_observations = data['recent_species_observations']
    ebird_token = data['ebird_token']
    gmap_api_key = data['gmap_api_key']
days_gap = input('Number of days from one sample to next, default to 3 ')
try:
    days_gap = int(days_gap)
except Exception as e:
    #print(e)
    days_gap = 3
recent_species_observations = recent_species_observations.format(back=days_gap*7)
recent_species_observations

Number of days from one sample to next, default to 3 


'https://api.ebird.org/v2/data/nearest/geo/recent/palwar?lat=38.90&lng=-77.07&back=21&includeProvisional=true&maxResults=3000'

In [39]:
import requests
headers = {}
headers['X-eBirdApiToken'] = ebird_token
response = requests.get(recent_species_observations, headers=headers)
res_json = response.json()

In [40]:
# res_json

In [50]:
import datetime as dt
import collections as col
# some observation date has no HH:MM part, just dates
DATE_FORMAT = '%Y-%m-%d'
Sighting = col.namedtuple('Sighting', ['lat', 'lng', 'date'])
sightings = list()
common_name = res_json[0]['comName']
for sighting in res_json:
    sightings.append(Sighting(lat=sighting['lat'], lng=sighting['lng'],
                              date=dt.datetime.strptime(sighting['obsDt'][:10], DATE_FORMAT)))
today = dt.date.today()
# 0th is yesterday, 1st is yesterday - days_gap days, 2nd is yesterday - 2*days_gap days, etc
# total length will be 7
# even with days_gap = 1 (no gap), today will not be included
sightings_last_seven_gapped_days = list()
for i in range(1, 7*days_gap, days_gap):
    sightings_last_seven_gapped_days.append(
        list(filter(lambda s: s.date.date() == today - dt.timedelta(days=i), sightings)))
#sightings_last_seven_gapped_days
common_name

'Palm Warbler'

In [42]:
layers = list()
yx_layers = list()
# colors of the rainbow, the more red the color, the closer to today
colors = list(['#ff0000', '#ffa500', '#ffff00', '#008000', '#0000ff', '#4b0082', '#ee82ee'])
for day_sightings in sightings_last_seven_gapped_days:
    layers.append(list(map(lambda s: (s.lat, s.lng), day_sightings)))
    yx_layers.append(list(map(lambda s: (s.lng, s.lat), day_sightings)))

In [56]:
import gmaps
gmaps.configure(api_key=gmap_api_key)
fig = gmaps.figure()
def add_layer(lat_lng, color, scale):
    layer = gmaps.symbol_layer(lat_lng, fill_color=color, stroke_color=color, scale=scale)
    fig.add_layer(layer)
# not counting yesterday, used to further limit the 7 days to a smaller number of days, if desired
#days_back = int(input('Number of days to show, 0 for today only '))
days_back = 4
days_back = min(max(0, days_back), 6)
# add the layers backwards so today is on top, but it does not seem to work,
# looks like there is no support for setting zIndex on a symbol (or symbols of a layer)
for i in range(days_back, -1, -1):
    add_layer(layers[i], colors[i], 2)
    # this line changes the size of the dots according to how recent they are
    #add_layer(layers[i], colors[i], (4 - i) if (days_back <= 3) else int((8-i)/2))
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
%matplotlib inline

In [59]:
%%capture
# turn off output for this cell so that we don't get a standalone map, we just need to see the animation at the end
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gp
from matplotlib import animation, rc
#from IPython.display import HTML

fig, ax = plt.subplots(figsize=(9, 7))
ax.set_xlim((-100, -70))
ax.set_ylim((22, 42))

first_day = days_gap * days_back + 1

plt.title(f"{common_name} ebird sightings every {days_gap} days from {first_day} days ago to yesterday (red)")

states = gp.read_file('../data/geo/us-census/cb_2018_us_state_5m.shp')
states.boundary.plot(ax=ax, color='#bbbbbb')

# sp for scatter plot
sp = ax.scatter([], [])

In [60]:
def update_frame(day, start, layers, colors, sp):
    #print(day)
    if day - start > 0:
        # do nothing if we should not start animating yet,
        # see the comment about pause below
        sp.set_offsets(np.empty((0, 2)))
        return sp,
    if day < 0:
        # let the last screen of red dots linger a bit longer
        return sp,
    sp.set_offsets(np.array(yx_layers[day]))
    sp.set_color(colors[day])
    return sp,
# pause is the first few frames that intentionally leave blank
# (and last few frames that stays with the red dots)
# so that when the animation runs in a loop there will be a clearer start and end
pause = 2
ani = animation.FuncAnimation(fig, update_frame, range(days_back+pause, -1-pause, -1), fargs=(days_back, layers, colors, sp),
                              interval=500, blit=True)

#HTML(ani.to_html5_video())
rc('animation', html='html5')
ani